<a href="https://colab.research.google.com/github/abisubramanya27/CS6910_Assignment2/blob/main/partA/src/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
# Mounting drive to store dataset
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [47]:
%cd gdrive/MyDrive/assignments/cs6910/A2/Data
!pwd

[Errno 2] No such file or directory: 'gdrive/MyDrive/assignments/cs6910/A2/Data'
/content/gdrive/.shortcut-targets-by-id/1H4LUGHYi_ivI7p5xWjyrjFojgIGEh9Jd/cs6910/A2/Data
/content/gdrive/.shortcut-targets-by-id/1H4LUGHYi_ivI7p5xWjyrjFojgIGEh9Jd/cs6910/A2/Data


In [48]:
# !pip install split-folders

In [49]:
# import splitfolders

# splitfolders.ratio('./inaturalist_12K/train', output='./inaturalist_12K/output', seed=1337, ratio=(.9, .1), group_prefix=None)

In [55]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [56]:
# Hyperparameters for building the model for Part - A
K_list_1 = [32, 32, 32, 32, 64, 64, 64, 64, 128, 128]       # List of number of filters in each non FC layer
F_list_1 = [11, 3, 5, 3, 3, 3, 3, 3, 3, 3]                  # List of size of filters in each non FC layer  
no_neurons_dense_1 = 1024                                   # Number of neurons in the dense FC layer
activation_fn_list_1 = ['relu']*6                           # List of activation function in each convolution and FC layer
P_list_1 = ['valid']*10                                     # List of padding options in each non FC layer ('valid' : no padding, 'same' : padding to make input and output same dimensions)
S_list_1 = [4, 2, 1, 1, 1, 1, 1, 2, 1, 1]                   # List of number of strides in each convolution layer
inp_img_shape_1 = (227, 227, 3)                             # Shape of input image from data
no_classes_1 = 10                                           # Number of output classes in the classification problem

def build_model_partA(inp_img_shape, K_list, F_list, no_neurons_dense, no_classes = 10, activation_fn_list = ['relu']*6, 
                      P_list = ['valid']*10, S_list = [1]*10, BN_yes = False, dropout_p = 0):
    '''
    Function to build the model comprising (5 conv+relu+maxpooling layers + 1 dense FC layer) for part A in keras
    Arguments :
        inp_img_shape - shape of input image
    '''
    model = Sequential()
    # First layer
    model.add(Conv2D(filters = K_list[0], kernel_size = (F_list[0], F_list[0]), strides = (S_list[0], S_list[0]), 
                     padding = P_list[0], input_shape = inp_img_shape))
    if BN_yes:
        model.add(BatchNormalization())
    model.add(Activation(activation_fn_list[0]))
    model.add(MaxPooling2D(pool_size=(F_list[1], F_list[1]), strides = (S_list[1], S_list[1]), padding = P_list[1]))

    # 4 Conv-relu-MaxPooling layers
    for l in range(1, 5):
        model.add(Conv2D(filters = K_list[2*l], kernel_size = (F_list[2*l], F_list[2*l]), strides = (S_list[2*l], S_list[2*l]), 
                         padding = P_list[2*l], input_shape = inp_img_shape))
        if BN_yes:
            model.add(BatchNormalization())
        model.add(Activation(activation_fn_list[l]))
        model.add(MaxPooling2D(pool_size = (F_list[2*l+1], F_list[2*l+1]), strides = (S_list[2*l+1], S_list[2*l+1]), padding = P_list[2*l+1]))
    
    # 1 dense FC layer
    model.add(Flatten())
    model.add(Dropout(dropout_p))
    model.add(Dense(units = no_neurons_dense))
    if BN_yes:
        model.add(BatchNormalization())
    model.add(Activation(activation = activation_fn_list[5]))

    # Output layer
    model.add(Dense(units = no_classes))
    if BN_yes:
        model.add(BatchNormalization())
    model.add(Activation(activation = 'softmax'))

    return model


# PART - A, Question 1
modelA = build_model_partA(inp_img_shape_1, K_list_1, F_list_1, no_neurons_dense_1, no_classes_1, 
                           activation_fn_list_1, P_list_1, S_list_1, True, 0.2)
modelA.summary()


Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_62 (Conv2D)           (None, 55, 55, 32)        11648     
_________________________________________________________________
batch_normalization_5 (Batch (None, 55, 55, 32)        128       
_________________________________________________________________
activation_5 (Activation)    (None, 55, 55, 32)        0         
_________________________________________________________________
max_pooling2d_61 (MaxPooling (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 23, 23, 32)        25632     
_________________________________________________________________
batch_normalization_6 (Batch (None, 23, 23, 32)        128       
_________________________________________________________________
activation_6 (Activation)    (None, 23, 23, 32)      

In [57]:
!pip install --upgrade wandb
!wandb login 6746f968d95eb71e281d6c7772a0469574430408

  Using cached https://files.pythonhosted.org/packages/f9/64/29b77da604e81607e35479bda8c31aabe8911c284fdad488a9030fa4cc0a/wandb-0.10.23-py2.py3-none-any.whl
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [58]:
import wandb
from wandb.keras import WandbCallback

run = wandb.init(project="assignment2", entity="abisheks", reinit=True, config={
                    "learning_rate": 1e-3,
                    "epochs": 10,
                    "batch_size": 32,
                    "loss_function": 'categorical_crossentropy',
                    "architecture": 'CNN',
                    "dataset": "iNaturalist_12K"
                })
config = wandb.config

wandb: Currently logged in as: abisheks (use `wandb login --relogin` to force relogin)


In [61]:
train_gen_1 = ImageDataGenerator(rotation_range = 30, height_shift_range = 0.15, width_shift_range = 0.15, channel_shift_range = 10,
                               shear_range = 0.15, zoom_range = 0.2, horizontal_flip = True, vertical_flip = True).flow_from_directory(
                               './inaturalist_12K/train', target_size = (227, 227), batch_size = config.batch_size)
val_gen_1 = ImageDataGenerator().flow_from_directory('./inaturalist_12K/val', target_size = (227, 227), batch_size = config.batch_size)
test_gen_1 = ImageDataGenerator().flow_from_directory('./inaturalist_12K/test', target_size = (227, 227), batch_size = config.batch_size)

Found 9006 images belonging to 10 classes.
Found 1004 images belonging to 10 classes.
Found 2008 images belonging to 10 classes.


In [66]:
# Running sample run
tf.keras.backend.clear_session()
modelA.compile(optimizer = Adam(learning_rate=config.learning_rate), loss = config.loss_function, metrics = ['accuracy'])
modelA.fit(train_gen_1,
           epochs = config.epochs, 
           validation_data = val_gen_1,
           verbose = 2,
           callbacks = [WandbCallback()])


Epoch 1/10
282/282 - 304s - loss: 2.1985 - accuracy: 0.2175 - val_loss: 3.0946 - val_accuracy: 0.1444
Epoch 2/10
282/282 - 245s - loss: 2.0817 - accuracy: 0.2503 - val_loss: 2.5829 - val_accuracy: 0.1594
Epoch 3/10
282/282 - 244s - loss: 2.0324 - accuracy: 0.2760 - val_loss: 2.1773 - val_accuracy: 0.2161
Epoch 4/10
282/282 - 244s - loss: 2.0060 - accuracy: 0.2848 - val_loss: 2.1117 - val_accuracy: 0.2410
Epoch 5/10
282/282 - 244s - loss: 2.0005 - accuracy: 0.2903 - val_loss: 2.2991 - val_accuracy: 0.2002
Epoch 6/10
282/282 - 243s - loss: 1.9689 - accuracy: 0.3057 - val_loss: 2.1206 - val_accuracy: 0.2420
Epoch 7/10
282/282 - 243s - loss: 1.9553 - accuracy: 0.3088 - val_loss: 1.9812 - val_accuracy: 0.2978
Epoch 8/10
282/282 - 242s - loss: 1.9364 - accuracy: 0.3128 - val_loss: 1.9789 - val_accuracy: 0.2859
Epoch 9/10
282/282 - 243s - loss: 1.9216 - accuracy: 0.3188 - val_loss: 2.0697 - val_accuracy: 0.2709
Epoch 10/10
282/282 - 244s - loss: 1.9115 - accuracy: 0.3270 - val_loss: 2.2134 - 